In [1]:
# title: parse_data.ipynb
# author: @ericramsey
# description: file used to clean obtained data and format numerically for machine learning implimentation

import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")] # joins the file to the SCORE_DIR giving the full path to the html file

In [5]:
# function used to takes the path of a box_score file and cleans the data being retrieved, removing table columns that are not needed
def parse_html(box_score):
    
    with open(box_score) as f: # opens and reads box_score file
        
        html = f.read()
        
    soup = BeautifulSoup(html) # BeautifulSoup instance used to parse html
    [s.decompose() for s in soup.select("tr.over_header")] # selects the tr (table-row) tags from over_header and removes 
    [s.decompose() for s in soup.select("tr.thead")] # selects the tr (table-row) tags from thead and removes the Reference column causing visual error
    
    return soup

In [6]:
# function used to obtain information about the data retrieved - identifying the season from the bottom_nav_container
def read_season_info(soup):
    
    nav = soup.select("#bottom_nav_container")[0] # selects the bottom_nav_container
    hrefs = [a["href"] for a in nav.find_all('a')] # locates the links in nav instance
    season = os.path.basename(hrefs[1]).split("_")[0] # splits the first link from the "_" and retrieve first element
    
    return season

In [7]:
# function used to indicate how many points a team scored, winning team, using line_score data from url scrapped
def read_line_score(soup):
        
        line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0] # initializes line_score while using pandas to identify table id and retrive line_score data
        cols = list(line_score.columns) # converts the table column names into a list
        cols[0] = "team" # first column name
        cols[-1] = "total" # second column name
        line_score.columns = cols # assigns the established col id's back to line_score
        
        line_score = line_score[["team", "total"]] # removes quartly scores to prevent issues with games with overtime, returning only total scores
    
        return line_score

In [8]:
# function used to read the stats for each team
def read_stats(soup, team, stat):
    
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0] # parse tables for all teams and stat data retrieved
    df = df.apply(pd.to_numeric, errors="coerce") # converts columns to numeric columns to be used for machine learning
    
    return df

In [14]:
games = [] # list used to store game data
base_cols = None # initialize base_cols instance

for box_score in box_scores:
    
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"]) # obtains the team from line_score 

    summaries = []
    
    for team in teams:
        
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]]) # concats basic and advanced tables into a sincle column (required for ML)
        totals.index = totals.index.str.lower() # formats the column names into lowercase

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()]) # concats basic and advanced tables into a sincle column (required for ML) for max player stats
        maxes.index = maxes.index.str.lower() + "_max" # formats the column names into lowercase adding "_max" to identify seperate from totals

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            
            base_cols = list(summary.index.drop_duplicates(keep="first")) # removes any duplicate cols (minutes played - mp)
            base_cols = [b for b in base_cols if "bpm" not in b] # removes bpm from box scores - causes error due to not being contained in all scores
        
        summary = summary[base_cols]
        summaries.append(summary)
        
    summary = pd.concat(summaries, axis=1).T # concats data into a single summary table
    
    game = pd.concat([summary, line_score], axis=1) # concats the summary and line score data 
    game["home"] = [0,1] # assigns home and away teams (0 = home, 1 = away)

    game_opp = game.iloc[::-1].reset_index() # reverses the data frame for the opponent data - comparing both for ML Model
    game_opp.columns += "_opp" 

    full_game = pd.concat([game, game_opp], axis=1) # concats the establish game and game_opp data frames
    full_game["season"] = read_season_info(soup) # calls read_season_info() function to analyze
    
    full_game["date"] = os.path.basename(box_score)[:8] # retrives the first eight characters from path - returing the date
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d") # formates the game data to datetime formate "yyyy-mm-dd"
    
    full_game["won"] = full_game["total"] > full_game["total_opp"] # determines and identifies game winner based on total points
    games.append(full_game) # 
    # Displays the total num games processes vs the total num games retrieved for the status of this process
    if len(games) % 100 == 0:
        
        print(f"{len(games)} / {len(box_scores)}")

100 / 8867
200 / 8867
300 / 8867
400 / 8867
500 / 8867
600 / 8867
700 / 8867
800 / 8867
900 / 8867
1000 / 8867
1100 / 8867
1200 / 8867
1300 / 8867
1400 / 8867
1500 / 8867
1600 / 8867
1700 / 8867
1800 / 8867
1900 / 8867
2000 / 8867
2100 / 8867
2200 / 8867
2300 / 8867
2400 / 8867
2500 / 8867
2600 / 8867
2700 / 8867
2800 / 8867
2900 / 8867
3000 / 8867
3100 / 8867
3200 / 8867
3300 / 8867
3400 / 8867
3500 / 8867
3600 / 8867
3700 / 8867
3800 / 8867
3900 / 8867
4000 / 8867
4100 / 8867
4200 / 8867
4300 / 8867
4400 / 8867
4500 / 8867
4600 / 8867
4700 / 8867
4800 / 8867
4900 / 8867
5000 / 8867
5100 / 8867
5200 / 8867
5300 / 8867
5400 / 8867
5500 / 8867
5600 / 8867
5700 / 8867
5800 / 8867
5900 / 8867
6000 / 8867
6100 / 8867
6200 / 8867
6300 / 8867
6400 / 8867
6500 / 8867
6600 / 8867
6700 / 8867
6800 / 8867
6900 / 8867
7000 / 8867
7100 / 8867
7200 / 8867
7300 / 8867
7400 / 8867
7500 / 8867
7600 / 8867
7700 / 8867
7800 / 8867
7900 / 8867
8000 / 8867
8100 / 8867
8200 / 8867
8300 / 8867
8400 / 8867
8

In [10]:
games_df = pd.concat(games, ignore_index=True) # combines all games - treats the game data as individual rows

In [11]:
# [g.shape for g in games if g.shape[1] != 150] # used for trouble shooting to identify games have all been retrieved

In [12]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17729,240.0,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17730,240.0,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17731,240.0,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17732,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [13]:
games_df.to_csv("nba_games.csv") # writes the cleaned and formated data to a nba_games.csv file to be imported for ML